In [1]:
import numpy as np
import pdb
# import matplotlib
# matplotlib.use('Agg')
import matplotlib.pylab as plt
from scipy import special

from context import aep
from datautils import step, spiral
from context import config

In [2]:
import os
data_path = '/tmp'
train_file, test_file = [os.path.join(data_path,
                                      'rectangles_im_' + ds + '.amat') 
                         for ds in ['train','test']]

In [9]:
def load_data(filename):
    file = open(filename,'rt')
    x = np.array([line.split()[:-1] for line in file.readlines()], np.float64)
    file = open(filename,'rt')
    y = np.array([0 if line.split()[-1]=='0.0000000e+00' else 1 for line in file.readlines()], np.int32)
    return x,y

x_train, y_train = load_data(train_file)
x_test, y_test = load_data(test_file)
y_train = np.reshape(y_train,(-1,1))
y_test = np.reshape(y_test,(-1,1))
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

((12000, 784), (50000, 784))
((12000, 1), (50000, 1))


In [4]:
y_train[np.where(y_train == 0)[0]] = -1

In [ ]:
M = 100 #number of inducing points
hidden_size = [30,30] #dimensions of hidden layers
model = aep.SDGPR(x_train, y_train, M, hidden_size, lik='Probit')
mb_size = 1000 #mini-batch size
model.optimise(method='adam', alpha=1.0, maxiter=1000,
                mb_size=mb_size, adam_lr=0.001)

In [6]:
from scipy import special
def inv_probit(x):
    jitter = 1e-3  # ensures output is strictly between 0 and 1
    return 0.5 * (1.0 + special.erf(x / np.sqrt(2.0))) \
        * (1 - 2 * jitter) + jitter
def assess_model_dgp(model, X_batch, Y_batch):
    mf, vf = model.predict_f_mm_linearMean(X_batch)
    p = inv_probit(mf / np.sqrt(1 + vf))
    a = ((p>0.5).astype(int) == Y_batch.astype(int))
    l = np.log(np.where(np.equal(Y_batch, 1), p, 1 - p))
    return l,a

In [7]:
def batch_assess(model, assess_model, X, Y):
    n_batches = max(int(len(X)/1000), 1)
    lik, acc = [], []
    for X_batch, Y_batch in zip(np.split(X, n_batches), np.split(Y, n_batches)):
        l, a = assess_model(model, X_batch, Y_batch)
        lik.append(l)
        acc.append(a)
    lik = np.concatenate(lik, 0)
    acc = np.array(np.concatenate(acc, 0), dtype=float)
    return np.average(lik), np.average(acc)

In [48]:
l,a = batch_assess(model, assess_model_dgp, x_test, y_test)
print('aep test lik: {:.4f}, test acc {:.4f}'.format(l, a))

aep test lik: -0.4803, test acc 0.7546
